In [26]:
import json
import numpy as np

In [77]:
with open("../../datasets/AAPD/llm_cluster_result/zero_shot_text_train.jsonl","r") as json_file:
    json_list = list(json_file)

In [78]:
acc_list = np.zeros(len(json_list))
for i,row in enumerate(json_list):
    acc_list[i] = float(json_list[i].split('"scores": ')[1][1:].split(', ')[0])

In [79]:
rank_list = np.argsort(acc_list)[:500]

In [80]:
file1 = open('../../datasets/AAPD/llama_label_50.txt', 'r')
document = file1.readlines()[:11037]  

In [81]:
documents = []
for index in rank_list:
    documents.append(document[index])

In [82]:
file1 = open('../../datasets/AAPD/train_texts_split_50.txt', 'r')
docs = file1.readlines()[:11037]
original_docs = []
for index in rank_list:
    original_docs.append(docs[index])  


In [52]:
import os
import sklearn.cluster
import numpy as np
from sklearn.mixture import GaussianMixture
from InstructorEmbedding import INSTRUCTOR
from numpy import savetxt
import pandas as pd
from collections import defaultdict
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 
os.environ['OPENBLAS_NUM_THREADS'] = '1'

cluster_num = 50

/home/xil240/anaconda3/envs/Multi-Label/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [66]:
docs = []
instruction = "Represent documents collected from arxiv abstract for clustering: "
for line in documents:
    new_line = line.split(": ")[1].strip()
    docs.append([instruction, new_line])


model = INSTRUCTOR('hkunlp/instructor-large')
embeddings = model.encode(docs)
print(embeddings)
print(embeddings.shape)

from umap import UMAP

umap_model = UMAP(n_neighbors=5, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
import numpy as np
umap_model.fit(embeddings)
umap_embeddings = np.nan_to_num(umap_model.transform(embeddings))

clustering_model = GaussianMixture(n_components = cluster_num, random_state=0)
clustering_model.fit(umap_embeddings)
predict_label = clustering_model.predict(umap_embeddings)
means = clustering_model.means_

# data = {'Document': original_docs, 
#         'Label': predict_label} 
# df = pd.DataFrame(data) 

# df.to_csv("../datasets/Reuters-21578/labelSpace/GMM_doc_label_50_test.csv", index=False)

select_docs_per_label = []
for label in range(cluster_num):
    embedding_list = []
    embedding_index = []
    cur_mean = means[label]
    for i, predict in enumerate(predict_label):
        if predict == label:
            embedding_list.append(umap_embeddings[i])
            embedding_index.append(i)
    dis_array = np.zeros(len(embedding_list))
    for index, embed in enumerate(embedding_list):
        distance = np.linalg.norm(embed - cur_mean)
        dis_array[index] = distance
    sort_indexs = np.argsort(dis_array)[:3]
    doc_list = ''
    for cur_index in sort_indexs:
        real_index = embedding_index[cur_index]
        doc_list += original_docs[real_index].strip() + ' '
    select_docs_per_label.append(doc_list)

    if label % 50 == 0:
        print(label)

with open('../datasets/AAPD/llm_cluster_result/doc_cluster_sec_round.txt', 'a') as the_file:
     for content in select_docs_per_label:
        the_file.write(content + '\n')

load INSTRUCTOR_Transformer
max_seq_length  512
[[-0.0199602   0.00421679 -0.00083727 ... -0.03149104  0.05162492
   0.06221992]
 [-0.02482701 -0.00586643 -0.01467834 ... -0.05478743  0.00761514
   0.03603554]
 [-0.01608321  0.00441551  0.00027117 ... -0.05026682  0.0122356
   0.0833294 ]
 ...
 [-0.04761259  0.03474179  0.00177428 ...  0.00370439  0.00936694
   0.06089918]
 [-0.01210953  0.00297768  0.02156829 ... -0.01119911  0.03135183
   0.07333915]
 [-0.05901276 -0.00917071 -0.01007354 ... -0.04394393  0.00919303
   0.0564457 ]]
(1000, 768)


/home/xil240/anaconda3/envs/Multi-Label/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


0


FileNotFoundError: [Errno 2] No such file or directory: '../datasets/AAPD/llm_cluster_result/doc_cluster_sec_round.txt'

In [67]:
with open('../../datasets/AAPD/llm_cluster_result/doc_cluster_sec_round.txt', 'a') as the_file:
     for content in select_docs_per_label:
        the_file.write(content + '\n')

In [69]:
import os
import numpy as np
from transformers import AutoTokenizer
import transformers
import torch

file1 = open('../../datasets/AAPD/llm_cluster_result/doc_cluster_sec_round.txt', 'r')
documents = file1.readlines()

system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

#Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- the relation between pearson 's correlation coefficient and salton 's cosine measure is revealed based on the different possible values of the division of the l1 norm and the l2 norm of a vector 
- these different values yield a sheaf of increasingly straight lines which form together a cloud of points , being the investigated relation the theoretical results are tested against the author co citation relations 
- among 24 informetricians for whom two matrices can be constructed , based on co citations the asymmetric occurrence matrix and the symmetric co citation matrix both examples completely confirm the theoretical results 
- the results enable us to specify an algorithm which provides a threshold value for the cosine above which none of the corresponding pearson correlations would be negative using this threshold value can be expected to optimize the visualization of the vector space"

Based on the information about the topic above, please find one label for this topic above. Make sure you only return the output without anything else in one line.

[/INST] Information Retrieval
"""

# example_prompt = """
# I have a topic that contains the following documents:
# - OHIO MATTRESS &lt;OMT> MAY HAVE LOWER 1ST QTR NET Ohio Mattress Co said its first quarter, ending February 28, profits may be below the 2.4 mln dlrs, or 15 cts a share, earned in the first quarter of fiscal
# - 1986. The company said any decline would be due to expenses related to the acquisitions in the middle of the current quarter of seven licensees of Sealy Inc, as well as 82 pct of
# - the outstanding capital stock of Sealy. Because of these acquisitions, it said, first quarter sales will be substantially higher than last year's 67.1 mln dlrs. Noting that it typically reports first quarter results in
# - late march, said the report is likely to be issued in early April this year. It said the delay is due to administrative considerations, including conducting appraisals, in connection with the acquisitions.
  
# Based on the information about the topic above, please find one label for this topic above. Please output your answer use the following format in one line:

# [/INST] acquisitions
# """

# example_prompt = """
# I have a topic that contains the following documents:
# - omron hem 790it automatic blood pressure monitor with advanced omron health management software so far this machine has worked well
# - and is very simple to use . it is nice to have immediate feedback on the bloodpressure effects of my various exercises , 
# - food consumption , and relaxation or stress levels .

# Based on the information about the topic above, please find one label for this topic above. Please output your answer use the following format in one line:

# [/INST] health_personal_care
# """

main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

Based on the information about the topic above, please find one label for this topic above. Make sure you only return the output without anything else in one line.
[/INST]
"""

prompt = system_prompt + example_prompt + main_prompt

# Hugging face repo name
model = "meta-llama/Llama-2-13b-chat-hf" #chat-hf (hugging face wrapper version)

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map='auto' # if you have GPU
)

with open('../../datasets/AAPD/llm_cluster_result/50chunk_labelspace_secround.txt', 'a') as the_file:
    for i, doc in enumerate(documents):
        replaced_text = prompt.replace('[DOCUMENTS]', doc.strip())
        sequences = pipeline(
            replaced_text,
            do_sample=True,
            top_k=10,
            top_p = 0.9,
            temperature = 0.2,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )
        label_text = sequences[0]["generated_text"][1250:]
        index = label_text.find('[/INST]') + len('[/INST] ')
        if label_text[index] == '\n':
            index += 1
        the_file.write(str(i) + ": " + label_text[index:] + '\n')
        print(str(i) + ": " + label_text[index:])

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

0: Database Repair and Optimization
1: Communication Networks
2: Information Retrieval
3: Optimization
4: Digital Image Forensics
5: Social Network Analysis
6: Formal Language Theory
7: Information Theory
8: Communication Networks
9: Vehicular Routing and Localization


/home/xil240/anaconda3/envs/Multi-Label/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


10: Economics
11: Signal Processing
12: Statistical Modeling
13: Data Mining and Clustering
14: Communication Networks
15: Epidemiology and Public Health
16: Information Theory
17: Computational Complexity
18: Millimeter Wave MIMO Systems
19: Data Mining and Machine Learning
20: Collaborative Intelligence
21: Anomaly Detection in Time Series
22: Network Topology
23: Machine Learning
24: Computational Complexity Theory
25: Optimal Power Flow

Please let me know if you would like me to label another topic for you.
26: Data Analytics
27: Simulation and Modeling
28: Machine Learning
29: Information Theory
30: Theoretical Computer Science
31: Collaborative Filtering
32: Sensor Networks and Localization
33: Computational Fluid Dynamics
34: Signal Processing
35: Neural Networks
36: Information Theory
37: Information Theory
38: Network Analysis
39: Wireless Communications
40: Educational Psychology
41: Communication Systems

Please let me know if you would like me to label any other topics or 

In [70]:
file1 = open('../../datasets/AAPD/llm_cluster_result/update_labelspace.txt', 'r')
documents = file1.readlines()  

label_space = []
for row in documents:
    label = row.strip()
    label_space.append(label)

In [71]:
file1 = open('../../datasets/AAPD/llm_cluster_result/50chunk_labelspace_secround.txt', 'r')
documents = file1.readlines()
new_space = []
for row in documents:
    new_space.append(row.strip().split(': ')[1])
new_space

['Database Repair and Optimization',
 'Communication Networks',
 'Information Retrieval',
 'Optimization',
 'Digital Image Forensics',
 'Social Network Analysis',
 'Formal Language Theory',
 'Information Theory',
 'Communication Networks',
 'Vehicular Routing and Localization',
 'Economics',
 'Signal Processing',
 'Statistical Modeling',
 'Data Mining and Clustering',
 'Communication Networks',
 'Epidemiology and Public Health',
 'Information Theory',
 'Computational Complexity',
 'Millimeter Wave MIMO Systems',
 'Data Mining and Machine Learning',
 'Collaborative Intelligence',
 'Anomaly Detection in Time Series',
 'Network Topology',
 'Machine Learning',
 'Computational Complexity Theory',
 'Optimal Power Flow',
 'Data Analytics',
 'Simulation and Modeling',
 'Machine Learning',
 'Information Theory',
 'Theoretical Computer Science',
 'Collaborative Filtering',
 'Sensor Networks and Localization',
 'Computational Fluid Dynamics',
 'Signal Processing',
 'Neural Networks',
 'Informatio

In [72]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device = 0)

In [84]:
for index in range(len(new_space)):
    query_embedding = model.encode(new_space[index])
    #print(new_space[index])
    passage_embedding = model.encode(label_space)
    sim_scores = util.dot_score(query_embedding, passage_embedding)[0].numpy()
    rank_list = np.argsort(sim_scores)[-1]
    if sim_scores[rank_list] <= 0.45:
        label_space.append(new_space[index])

In [85]:
with open('../../datasets/AAPD/llm_cluster_result/update_labelspace1.txt', 'a') as the_file:
    for label in label_space:
        the_file.write(label + '\n')